Citation

In [1]:
import ee,json,os,colour,math,csv,geemap
import numpy as np
import geopandas as geopandas
import pandas as pd
import datetime
ee.Authenticate()
ee.Initialize()
import time
'''Tested other values for cloud prob'''
#https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless

Enter verification code: 4/1AfgeXvsM_nE0ekyDji4hiK3h5s_lIZNd0LfJuWadtwB2JcS3KsJng16xeDk

Successfully saved authorization token.


'Tested other values for cloud prob'

### Variables

In [236]:
site='d3'
in_shp = r'E:\GIS\Thesis\Intermediate Data\Z_AOIs\{}_AOI.shp'.format(site)
Start_date,End_Date="2019-08-05","2019-08-28"
#Start_date,End_Date="2022-08-01","2022-08-31"
sen2= ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

#Cloud variables for detecting clouds and deciding tolerences
#MAX_CLOUD_PROBABILITY = 10
MAX_CLOUD_PROBABILITY = 50
#Cloudy_Percentage is how much cloudyness you are ok with in your mosaics.
#Set to very low value
#Cloudy_Percentage=.01
Cloudy_Percentage=.1

In [237]:
def shift_time(start_date, day_increment):
    date_1 = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    end_date = date_1 + datetime.timedelta(days=day_increment)
    return date_1.strftime("%Y-%m-%d"),end_date.strftime("%Y-%m-%d")
def mask_to_cloud(image):
    #Isolates the clouds in an image
    qa = image.select('MSK_CLDPRB')
    cloud = qa.lt(MAX_CLOUD_PROBABILITY)
    return image.updateMask(cloud) 

In [238]:
aoi = geemap.shp_to_ee(in_shp)
L_Dates = []
start = datetime.datetime.strptime(Start_date, "%Y-%m-%d")
#Pix_count_date is a date series used to calculate how many pixels in AOI
Pix_count_date=start + datetime.timedelta(days=15)
end = datetime.datetime.strptime(End_Date, "%Y-%m-%d")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days,2)]
for date in date_generated:
    #Creates a list of Dates within start_date and end_date
    L_Dates.append(date.strftime("%Y-%m-%d"))

C:\code\envs\geemap\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


#### Determining number of Pixels in AOI and AOI Cloudy Threshold

In [239]:
#Create a mosaic of Sen2 Pix in AOI
full_isle = sen2.filterBounds(aoi) \
                                    .filter(ee.Filter.date(Start_date,Pix_count_date))
full_img=full_isle.mosaic()
#Count number of pixels in AOI
count_full = full_img.reduceRegion(**{'reducer': ee.Reducer.count(),'geometry': aoi.geometry(),'scale': 10,'maxPixels': 1e9})
count_full=count_full.get('B2').getInfo()
#Calculate AOI Cloudy Threshold
cloud_thresh=int(count_full*Cloudy_Percentage)
print("{} is the number of B2 pixels in the AOI.\n{} is the Cloudy Threshold".format(count_full,cloud_thresh))

2722 is the number of B2 pixels in the AOI.
272 is the Cloudy Threshold


In [240]:
start = time.time()
selected_dates=[]
num_date_range=0
print('Testing all the dates in the summer date range')
for i in L_Dates:
    V=shift_time(i, 2)
    sen2_isles = sen2.filterBounds(aoi) \
                                    .filter(ee.Filter.date(V[0],V[1])) \
                                    .filterMetadata('CLOUD_COVERAGE_ASSESSMENT', 'less_than', 50)
    #Sort by latest image so if there are multiple in an overlap the latest one is chosen
    sen2_isles=sen2_isles.sort('system:time_start', False) 
    #imgnum is the number of images retreived using the given dates
    img_num=len(sen2_isles.getInfo()['features'])
    if img_num >0:
        print(' Testing a mosaic to see if complete')
        img=sen2_isles.mosaic()
        #Counting the number of pixels in the mosaic to see how they compare to the known full image
        count = img.reduceRegion(**{'reducer': ee.Reducer.count(),'geometry': aoi.geometry(),'scale': 10,'maxPixels': 1e9})
        count = count.get('B2').getInfo()
        if count > (count_full-1000):
            print('  Mosaic passed completeness test, now testing for clouds')
            #Create a mask which just shows the clouds
            sen2_isles_CM=sen2_isles.map(mask_to_cloud)
            img=sen2_isles_CM.mosaic()
            count = img.reduceRegion(**{'reducer': ee.Reducer.count(),'geometry': aoi.geometry(),'scale': 10,'maxPixels': 1e9})
            count_clouds = count.get('B2').getInfo()
            #Comparing cloudy pixels to the cloud threshold
            if count_clouds>count_full-cloud_thresh:
                print('   Mosaic passed, has very low to no cloud coverage')
                fet_name=sen2_isles.getInfo()['id']
                #Appending the optimal dates to a list
            selected_dates.append([V[0],V[1]])
            num_date_range+=1
            print ('{} to {} has {} images and coveres all of the AOI'.format(V[0],V[1],img_num))
        else:
            print (' Failed, due to clouds')
end = time.time()
print('{} number of date ranges taking {}sec to retreive'.format(num_date_range,str((end - start))))

Testing all the dates in the summer date range
 Testing a mosaic to see if complete
  Mosaic passed completeness test, now testing for clouds
   Mosaic passed, has very low to no cloud coverage
2019-08-07 to 2019-08-09 has 2 images and coveres all of the AOI
 Testing a mosaic to see if complete
  Mosaic passed completeness test, now testing for clouds
2019-08-15 to 2019-08-17 has 2 images and coveres all of the AOI
 Testing a mosaic to see if complete
  Mosaic passed completeness test, now testing for clouds
2019-08-17 to 2019-08-19 has 2 images and coveres all of the AOI
 Testing a mosaic to see if complete
  Mosaic passed completeness test, now testing for clouds
   Mosaic passed, has very low to no cloud coverage
2019-08-25 to 2019-08-27 has 2 images and coveres all of the AOI
 Testing a mosaic to see if complete
  Mosaic passed completeness test, now testing for clouds
   Mosaic passed, has very low to no cloud coverage
2019-08-27 to 2019-08-29 has 2 images and coveres all of the A

In [241]:
Map = geemap.Map()
Sen2vis = {'bands': ['B4', 'B3', 'B2'],'min': 0,'max': 3000,'gamma': 1.4,}
SHPViz ={'color': 'Blue' }
imcollist=[]
for Timeframe in range(num_date_range):
    isle = sen2.filterBounds(aoi) \
        .filter(ee.Filter.date(selected_dates[Timeframe][0],selected_dates[Timeframe][1]))  \
        .filterMetadata('CLOUD_COVERAGE_ASSESSMENT', 'less_than', 50)
    isle=isle.sort('system:time_start', False)
    #creation of an image collection list
    imcollist.append(isle)
    Map.addLayer(isle, Sen2vis, '{} image'.format(selected_dates[Timeframe][0]))
Map.addLayer(aoi, SHPViz, 'AOI')
Map.addLayerControl()
Map.centerObject(aoi,16)
Map

Map(center=[53.47919804442465, -64.85701659563988], controls=(WidgetControl(options=['position', 'transparent_…

In [242]:
imcollist
imnamelist=[]
for i in imcollist:
    img=(i.getInfo())
    #this only gets the info for first image. need to make iterator if there are multiple images in each image collection
    img_name=r'COPERNICUS/S2/'+(img['features'][0]['properties']['system:index'])
    imnamelist.append(ee.Image(img_name))
print(imnamelist)
combined_img=ee.ImageCollection.fromImages(imnamelist)
#System index is the GEE name for Sen 2 images
r'COPERNICUS/S2/'+(img['features'][0]['properties']['system:index'])
combined_img_meadian=combined_img.median()

[<ee.image.Image object at 0x0000022A1B6CA130>, <ee.image.Image object at 0x0000022A1B439400>, <ee.image.Image object at 0x0000022A1BE68B20>, <ee.image.Image object at 0x0000022A1BD9BFA0>, <ee.image.Image object at 0x0000022A1BD0AC10>]


In [244]:
#can used median for median reducer
Sen2vis = {'bands': ['B4', 'B3', 'B2'],'min': 0,'max': 3000,'gamma': 1.4,}
Map = geemap.Map()
closest_date=imcollist[3].mosaic()
#Map.addLayer(combined_img_meadian, Sen2vis, 'median')
Map.addLayer(closest_date, Sen2vis, 'closest')
Map.addLayerControl()
Map.centerObject(aoi,16)
Map

Map(center=[53.47919804442465, -64.85701659563988], controls=(WidgetControl(options=['position', 'transparent_…

In [245]:
# Download the data
#can also set to imcollist[img] (use closest date and set the list value)

#median_mosaic=combined_img_meadian.mosaic()
#closest_date=imcollist[4].mosaic()
aoi_g= aoi.geometry()

geemap.ee_export_image(closest_date, filename=r'E:\GIS\Thesis\Intermediate Data\{}\Sen2\Sen_{}.tif'.format(site,site), scale=10,region=aoi_g, file_per_band=True)
print('done')

Generating URL ...
Please wait ...
Data downloaded to E:\GIS\Thesis\Intermediate Data\d3\Sen2
done
